# Parsing de Logs avec Python

## Import de librairies

In [1]:
import elasticsearch
from elasticsearch import helpers
import datetime

/home/bigdatalover/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


## Parsing de Logs

On désire lire des Logs d'accès depuis un fichier, en extraire les informations utiles (URLs, type de requête, status de réponse...), puis structurer ces infos et les sauvegarder sous un index Elasticsearch (base de données avec fonctionnalités de moteur de recherche)

Ci-dessous, on lit (stream) le contenu du fichier de Logs et on le charge dans une liste où chaque élément est une ligne du fichier...

In [2]:
list_of_logs = []
with open("access.log", "r") as file:
    lines = file.readlines()

On instancie le client python Elasticsearch qui va permettre d,indexer les données depuis Python vers la base Elasticsearch. Préalablement, il faut avoir installé et démarré la base Elasticsearch (possible en mode un seul noeud)

In [3]:
ES_CLIENT = elasticsearch.Elasticsearch("localhost:9200", timeout=10000)

In [4]:
settings = {
    "mappings": {
        "logs": {
            "properties": {
                "IP": {"type": "keyword"},
                "Request": {"type": "keyword"},
                "Url": {"type": "keyword"},
                "Status": {"type": "keyword"},}}}}

On crée l'index où on veut stocker les Logs structurés avec les propriétés définies juste au dessus...

In [5]:
ES_CLIENT.indices.create(index="logs", ignore=400, body=settings)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'logs'}

Le code ci-dessous permet d'extraire les infos voulues en tokenisant chaque ligne de log (le pattern de splitting est l'espace " ") et identifiant les infos via leurs positions dans les tokens... <br>
On met également ces infos dans le bon format qui permet de les passer en indexation vers Elasticsearch...

In [11]:
actions = []
current_id = 1
for line in lines[1:]:
    line_split = line.split(" ")
    action = {
        "_op_type": "update",
        "_index": "logs",
        "_type": "logs",
        "_id": current_id,
        "doc" : {
            "Date": datetime.datetime.strptime(
                "/".join(
                    [line_split[3].split("/")[1],
                     (line_split[3].split("/")[2]).split(":")[0]]), '%b/%Y'),
            "IP": line_split[0],
            "Request": line_split[5][1:],
            "Url": line_split[6],
            #"Status": int(line_split[8])
        },
        "doc_as_upsert" : True
    }
    try:
        action["doc"]["Status"] = int(line_split[8])
    except:
        pass
    actions.append(action)
    current_id += 1

In [12]:
actions

[{'_op_type': 'update',
  '_index': 'logs',
  '_type': 'logs',
  '_id': 1,
  'doc': {'Date': datetime.datetime(2015, 12, 1, 0, 0),
   'IP': '109.169.248.247',
   'Request': 'GET',
   'Url': '/administrator/',
   'Status': 200},
  'doc_as_upsert': True},
 {'_op_type': 'update',
  '_index': 'logs',
  '_type': 'logs',
  '_id': 2,
  'doc': {'Date': datetime.datetime(2015, 12, 1, 0, 0),
   'IP': '109.169.248.247',
   'Request': 'POST',
   'Url': '/administrator/index.php',
   'Status': 200},
  'doc_as_upsert': True},
 {'_op_type': 'update',
  '_index': 'logs',
  '_type': 'logs',
  '_id': 3,
  'doc': {'Date': datetime.datetime(2015, 12, 1, 0, 0),
   'IP': '46.72.177.4',
   'Request': 'GET',
   'Url': '/administrator/',
   'Status': 200},
  'doc_as_upsert': True},
 {'_op_type': 'update',
  '_index': 'logs',
  '_type': 'logs',
  '_id': 4,
  'doc': {'Date': datetime.datetime(2015, 12, 1, 0, 0),
   'IP': '46.72.177.4',
   'Request': 'POST',
   'Url': '/administrator/index.php',
   'Status': 200}

Enfin, insère tout le batch de Logs dans Elasticsearch via la méthode <b> helpers.bulk() </b>

In [13]:
helpers.bulk(ES_CLIENT, actions)

(26311, [])